# 🚛 Power BI Dashboard: Least Cost Carrier (LCC) Freight Optimization

## 📘 Overview
This Power BI solution helps identify least-cost freight carriers across customer shipments by comparing actual shipping costs with optimized LCC recommendations. The goal is to drive cost savings and increase visibility into freight spend by carrier, destination, and order.

## 🧩 Project Architecture
- **DALCO_DATA**: Ingested from distributor `.CSV` files dropped via FTP → loaded into Azure Data Lake → cleaned and appended to Power BI via the `DALCO_DATA` table.
- **LCC_DATA**: Contains LCC recommendations and optimized cost data. Used for calculating savings potential.
- **CUST_CUST_PO**: Pulled directly from Global Shop ERP (Pervasive v13 SQL Server) and joined via `CARRIER_CD`.

## 🔄 Data Flow Summary
1. Distributor sends CSV to FTP
2. Automated process loads CSV into Data Lake
3. Data is cleaned and stored in `DALCO_DATA`
4. `LCC_DATA` and `CUST_CUST_PO` tables joined via relationships
5. Power BI data model combines all sources for analysis and visuals

## 📊 Key Dashboard Features
- **Least Cost Carrier Opportunity Gauge**: Breakdown of shipments where savings were possible vs. not.
- **Treemap of Savings by LCC Carrier**: Identifies top carriers offering the most savings (e.g., USF Holland, R&L).
- **Geo Map**: Destination-based view of savings opportunities across North America and Europe.
- **Savings Table**: Row-level detail of freight spend, total cost, optimized cost, and savings per shipment.

## 💡 Business Value
- $109K+ in potential savings identified across active shipments.
- Improved visibility into carrier selection and freight cost patterns.
- Actionable insights for logistics and procurement teams to renegotiate carrier contracts and shipping methods.
- Automated data flow reduces manual updates and ensures data accuracy.

## 🔧 Tech Stack
| Layer        | Tools/Platforms |
|--------------|-----------------|
| Data Source  | CSV (FTP), SQL (Pervasive v13) |
| Storage      | Azure Data Lake |
| ETL/Modeling | Power Query (in Power BI) |
| BI Layer     | Power BI Desktop |
| Automation   | FTP Trigger → Lake Load Process |

## 📂 Data Tables
| Table         | Description |
|---------------|-------------|
| `DALCO_DATA`  | Freight-level detail from CSV files |
| `LCC_DATA`    | Least Cost Carrier recommendations and savings |
| `CUST_CUST_PO`| Customer PO and shipping records from ERP |

## 📌 Next Steps
- Integrate carrier API feeds for real-time LCC validation
- Add KPI scorecard for % of shipments using LCC
- Automate full ETL pipeline using Azure Data Factory

## 🧠 SQL Queries: Freight Optimization & GSS Data Extraction

### 1️⃣ Freight Spend & LCC Comparison by Order
This query pulls order-level shipping data and compares actual freight costs to optimized least-cost carrier values.

In [ ]:
-- Freight spend and least-cost comparison by carrier and order
SELECT
    po.ord_no AS order_number,
    po.customer AS customer_code,
    po.ship_via AS ship_method,
    po.carrier_cd AS carrier_used,
    lcc.lcc_carrier AS least_cost_carrier,
    po.date_order AS order_date,
    po.date_shipped,
    po.destination_city,
    po.destination_state,
    lcc.total_cost_carrier AS actual_cost,
    lcc.lcc_total_cost AS optimized_cost,
    (lcc.total_cost_carrier - lcc.lcc_total_cost) AS potential_savings,
    CASE 
        WHEN lcc.total_cost_carrier > lcc.lcc_total_cost THEN 'Yes'
        ELSE 'No'
    END AS lcc_opportunity
FROM
    CUST_CUST_PO po
LEFT JOIN
    LCC_DATA lcc
    ON po.ord_no = lcc.order_no
WHERE
    po.date_order >= '2025-01-01'
ORDER BY
    potential_savings DESC;

### 2️⃣ Aggregate Savings by Carrier
This query summarizes potential savings by least-cost carrier.

In [ ]:
-- Aggregate potential savings by LCC carrier
SELECT
    lcc.lcc_carrier,
    COUNT(*) AS total_shipments,
    SUM(lcc.total_cost_carrier - lcc.lcc_total_cost) AS total_savings,
    AVG(lcc.total_cost_carrier - lcc.lcc_total_cost) AS avg_savings_per_shipment
FROM
    LCC_DATA lcc
INNER JOIN
    CUST_CUST_PO po
    ON lcc.order_no = po.ord_no
WHERE
    lcc.total_cost_carrier > lcc.lcc_total_cost
GROUP BY
    lcc.lcc_carrier
ORDER BY
    total_savings DESC;

### 3️⃣ City-Level Freight Optimization
This pulls savings summaries by shipping destination.

In [ ]:
-- Savings by destination city
SELECT
    po.destination_city,
    COUNT(*) AS total_orders,
    SUM(lcc.total_cost_carrier - lcc.lcc_total_cost) AS city_savings
FROM
    LCC_DATA lcc
JOIN
    CUST_CUST_PO po
    ON lcc.order_no = po.ord_no
WHERE
    lcc.total_cost_carrier > lcc.lcc_total_cost
GROUP BY
    po.destination_city
ORDER BY
    city_savings DESC;